# **Getting started**

This guide will walk you through the creation and configuration of a new model on the platform - giving you model observability in 5 minutes!

[1. Import Superwise package](#Import-Superwise-package)

[2. Simulate model training flow](#Simulate-model-training-flow)

[3. Let's integrate with Superwise](#Let's-integrate-with-Superwise)
- [3.1 Create a Model](#Create-a-Model)
- [3.2 Create a Version](#Create-a-Version)
- [3.3 Log production data](#Log-production-data)



#### ***Prerequisite packages ( optional )***



In [ ]:
!pip install pandas numpy sklearn superwise

## **Import Superwise package**

The Superwise SDK is a standard Python package that simplifies the integration with Superwise and streams data to the Superwise platform.

For demostration purpose we will use:
* pandas for loading our diamonds data, working with dataframes
* numpy for dataframe operations
* sklearn for default ML operations such as train-test split, encoders, model trainning etc

Clarifications:
* Superwise.models.model.Model - represents a ML model in production
* Superwise.models.version.Version - represents a particular model form, with baseline data and features
* DataEntityRole - describes the dtype and role of the feature

In [1]:
import os

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from superwise import Superwise
from superwise.models.model import Model
from superwise.models.version import Version
from superwise.resources.superwise_enums import DataEntityRole

## Initiate Superwise client

There are 2 environment variables used to identify and authenticate your connection:
* SUPERWISE_CLIENT_ID
* SUPERWISE_SECRET

Read [here](https://docs.superwise.ai/docs/authentication) to understand how to
 generate them.


In [2]:
os.environ['SUPERWISE_CLIENT_ID'] = 'REPLACE_WITH_YOUR_CLIENT'
os.environ['SUPERWISE_SECRET'] = 'REPLACE_WITH_YOUR_SECRET'

To interact with the Superwise APIs, create an instance of the Superwise object. All APIs will now be accessible under the sw instance.


In [5]:
sw = Superwise()

## Simulate model training flow

For this tutorial, we use the Diamonds public dataset. This classic dataset contains the prices and other attributes of almost 54,000 diamonds.


#### Load data

In [6]:
diamonds = pd.read_csv('https://www.openml.org/data/get_csv/21792853/dataset')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Dataset properties:
- carat weight of the diamond (0.2--5.01)
- cut quality of the cut (Fair, Good, Very Good, Premium, Ideal)
- color diamond color, from J (worst) to D (best)
- clarity a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
- depth total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
- table width of top of diamond relative to widest point (43--95)
- price price in US dollars ($326--$18,823) - Label
- x length in mm (0--10.74)
- y width in mm (0--58.9)
- z depth in mm (0--31.8)


#### Split Train-Test

We’ll split the dataset into training and test sets. The training dataset will be used later as a reference dataset for the baseline. The test dataset will simulate production data that feeds into the model. You can read more about the baseline dataset concept [here](https://docs.superwise.ai/docs/baseline)


In [7]:
X = diamonds.drop(columns="price")
y = diamonds["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
19497,1.21,Ideal,H,VVS2,61.3,57.0,6.92,6.87,4.23
31229,0.31,Ideal,E,VS2,62.0,56.0,4.38,4.36,2.71
22311,1.21,Ideal,E,VS1,62.4,57.0,6.75,6.83,4.24
278,0.81,Ideal,F,SI2,62.6,55.0,5.92,5.96,3.72
6646,0.79,Ideal,I,VVS2,61.7,56.0,5.94,5.95,3.67


#### Pre-processing

To use categorical features in our model, we will first transform them into numerical values using one-hot encoding

In [8]:
categorical_cols = ['cut', 'clarity', 'color']

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical',  OneHotEncoder(), categorical_cols)
    ], remainder='passthrough')

#### Train your model

In this example, we train a simple Liner Regression

In [9]:
diamond_price_model=LinearRegression()

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', diamond_price_model)
])

my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['cut', 'clarity',
                                                   'color'])])),
                ('model', LinearRegression())])

#### Predict

We can now use the model we trained to predict the price


In [10]:
y_pred_train =  my_pipeline.predict(X_train)

## **Let's integrate with Superwise**

Now that we have a trained model, we can begin by logging it into the Superwise platform and starting to monitor its behavior.


#### Create a Model

Models, or machine learning-based decision processes are the basic atomic component that Superwise observes

In [11]:
diamond_model = Model(
    name="Diamond Model",
    description="Regression model that predicts the diamond price"
)

diamond_model = sw.model.create(diamond_model)
print(f"New model Created - {diamond_model.id}")

New model Created - 45


In [29]:
%%html
<img src="../images/new_model.png"/>

#### Create a Version

Deploying a model to production is only the first step, since models require continuous improvement and ongoing updates. The differences between versions may be ad hoc schema changes or retraining on a new data set to refit the model hyperparameters under the same given schema.

The process contains 3 main steps 
    - Prepare the baseline dataset based on the training data
    - Define the version schema
    - Log the version into the Superwise app

#### Prepare Baseline Dataset

We’re now going to add 4 columns to our dataset
* ID - Unique identifier per row or prediction (Required)
* Timestamp - Indicates when the prediction took place (Required)
* Model prediction
* Label - The real diamond price

In [12]:
baseline_data = X_train.assign(
    id=X_train.index,
    ts=pd.Timestamp.now(),
    prediction=y_pred_train,
    price=y_train
)
baseline_data["prediction"] = baseline_data["prediction"].astype(float)

data_dir = 'data'
baseline_data_filename = 'baseline.csv'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
baseline_data_filepath = os.path.join(data_dir, baseline_data_filename)  

baseline_data.to_csv(baseline_data_filepath, index=False)
baseline_data.head()

,carat,cut,color,clarity,depth,table,x,y,z,id,ts,prediction,price
19497,1.21,Ideal,H,VVS2,61.3,57.0,6.92,6.87,4.23,19497,2022-02-17 13:38:29.069049,8060.5,8131
31229,0.31,Ideal,E,VS2,62.0,56.0,4.38,4.36,2.71,31229,2022-02-17 13:38:29.069049,607.0,756
22311,1.21,Ideal,E,VS1,62.4,57.0,6.75,6.83,4.24,22311,2022-02-17 13:38:29.069049,8569.0,10351
278,0.81,Ideal,F,SI2,62.6,55.0,5.92,5.96,3.72,278,2022-02-17 13:38:29.069049,3031.0,2795
6646,0.79,Ideal,I,VVS2,61.7,56.0,5.94,5.95,3.67,6646,2022-02-17 13:38:29.069049,3862.0,4092


#### Infer schema

Because each model version can introduce new input formats, we must have an explicit schema definition for each version. The schema is a collection of different data entities (aka columns) that are part of the specific version of the machine learning decision process. Each data entity has its own data type and plays a specific role in the ML process.

You can read more about the roles of data entities and supported data types [here](https://docs.superwise.ai/docs/version)

In [13]:
entities_collection = sw.data_entity.summarise(
    data=baseline_data,
    specific_roles = {
      'id': DataEntityRole.ID,
      'ts': DataEntityRole.TIMESTAMP,
      'prediction': DataEntityRole.PREDICTION_VALUE,
      'price': DataEntityRole.LABEL
    }
)

Here are the schema’s main properties, which include roles, types, feature importance, and descriptive statistics:


In [14]:
ls = list()
for entity in entities_collection:
    ls.append(entity.get_properties())
    
pd.DataFrame(ls).head()

,data_type,dimension_start_ts,feature_importance,id,name,role,secondary_type,summary,type
0,number,None,61.53,None,carat,feature,Num_centered,"{'statistics': {'missing_values': 0.0, 'outlie...",Numeric
1,text,None,0.01,None,cut,feature,Cat_dense,"{'statistics': {'missing_values': 0.0, 'new_va...",Categorical
2,text,None,6.10,None,color,feature,Cat_dense,"{'statistics': {'missing_values': 0.0, 'new_va...",Categorical
3,text,None,11.15,None,clarity,feature,Cat_dense,"{'statistics': {'missing_values': 0.0, 'new_va...",Categorical
4,number,None,0.01,None,depth,feature,Num_centered,"{'statistics': {'missing_values': 0.0, 'outlie...",Numeric


#### Activate a version

Now that we have a model and schema, let’s combine them together into a version.


In [15]:
new_version = Version(
    model_id=diamond_model.id,
    name="1.0.0",
    data_entities=entities_collection,
)

new_version = sw.version.create(new_version)

In [16]:
sw.version.activate(new_version.id)

<Response [204]>

In [30]:
%%html
<img src="../images/new_version.png"/>

## Log production data

We’ll now use the test data to simulate production data.


#### Predictions

The prediction data should include all the data entities with the following roles - Id, Timestamp, Metadata, Feature, Prediction probability, Prediction value, Label weight -- as defined in the version schema above. In our diamonds example, we’ll add the following columns: Id, Timestamp, and Model prediction.
If some data entities are missing, you’ll get a schema skew error, which means that the data will not be streaming into Superwise.

In [17]:
y_test_pred= my_pipeline.predict(X_test)

In [18]:
prediction_time_vector = pd.Timestamp.now().floor('h') - \
    pd.TimedeltaIndex(X_test.reset_index(drop=True).index // int(X_test.shape[0] // 30), unit='D')

ongoing_predictions = X_test.assign(
    id=X_test.index,
    ts=prediction_time_vector,
    prediction=y_test_pred
)
ongoing_predictions["prediction"] = ongoing_predictions["prediction"].astype(float)
ongoing_predictions.to_csv('data/ongoing_predictions.csv', index=False)
ongoing_predictions.head()

,carat,cut,color,clarity,depth,table,x,y,z,id,ts,prediction
1388,0.24,Ideal,G,VVS1,62.1,56.0,3.97,4.00,2.47,1388,2022-02-17 13:00:00,712.0
50052,0.58,'Very Good',F,VVS2,60.0,57.0,5.44,5.42,3.26,50052,2022-02-17 13:00:00,3186.0
41645,0.40,Ideal,E,VVS2,62.1,55.0,4.76,4.74,2.95,41645,2022-02-17 13:00:00,1944.5
42377,0.43,Premium,E,VVS2,60.8,57.0,4.92,4.89,2.98,42377,2022-02-17 13:00:00,2081.0
17244,1.55,Ideal,E,SI2,62.3,55.0,7.44,7.37,4.61,17244,2022-02-17 13:00:00,9873.5


After we fit the data according to the schema, it’s ready to be sent to the Superwise platform. You’ll need to provide the model and the version of the model being used for the prediction.

Notice that each chunk of data should not be more than 1000 records.


In [19]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

While the next snippet is running, open [transactions](https://portal.superwise.ai) page within Superwise portal to see the data being loaded

In [32]:
ongoing_predictions_chunks = chunks(ongoing_predictions.to_dict(orient='records'), 1000)

transaction_ids = list()
for ongoing_predictions_chunk in ongoing_predictions_chunks:
    transaction_id = sw.transaction.log_records(
        model_id=diamond_model.id, 
        version_id=new_version.name, 
        records=ongoing_predictions_chunk
    )
    transaction_ids.append(transaction_id)
    print(transaction_id)

{'transaction_id': 'f45eaacc-8fe9-11ec-9adf-eea5408e5dc0'}
{'transaction_id': 'f552f0c8-8fe9-11ec-9adf-eea5408e5dc0'}
{'transaction_id': 'f629e4c0-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'f70d13ee-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'f7f19492-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'f8c5e940-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'f99df3c6-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'fa80101c-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'fb64daee-8fe9-11ec-b186-eea5408e5dc0'}
{'transaction_id': 'fc4badc0-8fe9-11ec-983e-eea5408e5dc0'}
{'transaction_id': 'fd2e45cc-8fe9-11ec-983e-eea5408e5dc0'}
{'transaction_id': 'fe13e6fe-8fe9-11ec-983e-eea5408e5dc0'}
{'transaction_id': 'ff078606-8fe9-11ec-983e-eea5408e5dc0'}
{'transaction_id': 'ffe096bc-8fe9-11ec-983e-eea5408e5dc0'}
{'transaction_id': '00c40910-8fea-11ec-983e-eea5408e5dc0'}
{'transaction_id': '01a9662c-8fea-11ec-b186-eea5408e5dc0'}
{'transaction_id': '02464a96-8fea-11ec-b186-eea5408e5dc0

In [36]:
 %%html
    <img src="images/transactions.png"/>

In [25]:
transaction_id = sw.transaction.get(transaction_id=transaction_ids[0]['transaction_id'])
transaction_id.get_properties()['status']

'Passed'

## Ground truth

In most ML scenarios, the ground truth comes back with a delay. To log it correctly, the data should include all the data entities with the following roles: Id, Label (as we defined in the schema). Superwise will automatically add them to the predictions that were sent earlier and use them to calculate the model performance metrics.

In [26]:
ongoing_labels = y_test.reset_index().copy().rename(columns={"index": "id"})
ongoing_labels.to_csv('data/ongoing_labels.csv', index=False)
ongoing_labels.head()

,id,price
0,1388,559
1,50052,2201
2,41645,1238
3,42377,1304
4,17244,6901


You’ll notice that when you log the ground truth, the version_id is optional. For example, if you use 2 different versions to predict the same data record, the ground truth is relevant for both of them.

In [27]:
ongoing_labels_chunks = chunks(ongoing_labels.to_dict(orient='records'), 1000)

transaction_ids = list()
for ongoing_labels_chunk in ongoing_labels_chunks:
    transaction_id = sw.transaction.log_records(
        model_id=diamond_model.id, 
        records=ongoing_labels_chunk
    )
    
    transaction_ids.append(transaction_id)
    print(transaction_id)

{'transaction_id': 'c5206b5e-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c585303e-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c5e90ed8-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c64d3c3c-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c6b2679c-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c724bd9c-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c78935e2-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c7ed02ac-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c85177fa-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c8b55edc-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c91884c6-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c9840c8c-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'c9e8de32-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'ca4d47e6-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'cab17fa4-8fe6-11ec-8af3-eea5408e5dc0'}
{'transaction_id': 'cb1568fc-8fe6-11ec-961c-eea5408e5dc0'}
{'transaction_id': 'cb628a74-8fe6-11ec-961c-eea5408e5dc0

In [28]:
transaction_id = sw.transaction.get(transaction_id=transaction_ids[0]['transaction_id'])
transaction_id.get_properties()['status']

'Passed'